In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
import numpy as np
import math
import random

In [4]:
import time

In [5]:
def show_state(env, episode,step, info=""):
    plt.figure(99999,figsize=[8,6])
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("episode: {} step: {} ".format(episode,step))
    #plt.title("%s | Step: %d %s" % (env._spec.id,step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

In [6]:
import gym

In [7]:
use_cuda = torch.cuda.is_available()
device=torch.device("cuda:5" if use_cuda else "cpu")

In [8]:
###### PARAMS ######
learning_rate = 0.001
num_episodes = 50000
gamma=0.999999
#gamma=0.85
egreedy = 0.9
egreedy_final = 0.01
egreedy_decay = 50000

report_interval=10

score_to_solve = 195

hidden_layer_size=64

replay_memory_size=500000

batch_size=32

update_target_frequency = 100000

clip_error=True

In [9]:
env = gym.make('Breakout-ramDeterministic-v4')

In [10]:
#env = gym.wrappers.Monitor(env, '../mp4/sandbox10',video_callable=lambda episode_id: True,force=True)
#env = gym.wrappers.Monitor(env, '../mp4/breakout_DQN_p1',video_callable=lambda episode_id: episode_id%100==0,force=True)

In [11]:
number_of_inputs=env.observation_space.shape[0]
number_of_outputs=env.action_space.n

In [12]:
def calculate_epsilon(steps_done):
    epsilon = egreedy_final + (egreedy - egreedy_final) * \
              math.exp(-1. * steps_done / egreedy_decay )
    return epsilon

In [13]:
class ExperienceReplay():
    def __init__(self, capacity):
        self.capacity=capacity
        self.memory=[]
        self.position=0
        
    
    def push(self, state,
             action, new_state,
             reward, done):
        
            transition=(state,action,new_state,reward,done)
            
            if self.position>=len(self.memory):
                self.memory.append(transition)
            else:
                self.memory[self.position]=transition
                
            self.position=(self.position+1)%self.capacity
        
    
    def sample(self,batch_size):
        return zip(*random.sample(self.memory, batch_size))
    
    def __len__(self):
        return len(self.memory)
    

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear1 = nn.Linear(number_of_inputs,hidden_layer_size)
        self.linear2 = nn.Linear(hidden_layer_size,number_of_outputs)
        
        self.activation=nn.Tanh()
        
    def forward(self, x):
        output1 = self.linear1(x)
        output1 = self.activation(output1)
        output2 = self.linear2(output1)
        
        return output2

In [15]:
class QNet_Agent():
    def __init__(self):
        
        self.nn = NeuralNetwork().to(device)
        self.target_nn = NeuralNetwork().to(device)
        
        self.loss_function = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.nn.parameters(), lr=learning_rate)
        
        self.update_target_counter = 0
        
    def select_action(self,state,epsilon):
        
        random_for_egreedy=torch.rand(1).item()
        
        if random_for_egreedy>epsilon:
            self.nn.eval()
            with torch.no_grad():
                state=torch.Tensor(state).to(device)
                predicted_value_from_nn=self.nn(state)
                action=torch.argmax(predicted_value_from_nn).item()
        else:
            action=env.action_space.sample()
                
                
        return action
    
    def optimize(self):
        
        if len(memory)<batch_size:
            return
        
        state, action, new_state, reward, done = memory.sample(batch_size)
        
        state=torch.Tensor(state).to(device)
        new_state=torch.Tensor(new_state).to(device)
        reward=torch.Tensor(reward).to(device)
        
        #the view call below is to transform into column vector
        #so that it can be used in the gather call
        #i.e. we will use it to pick out from the computed value
        #tensor only values indexed by selected action
        action=(torch.Tensor(action).view(-1,1).long()).to(device)
        #print('action: ')
        #print(action)
        #print('contiguous?', action.is_contiguous())
        done=torch.Tensor(done).to(device)
        
        #print('shape of: state, new state, reward, action, done:')
        #print(state.shape)
        #print(new_state.shape)
        #print(reward.shape)
        #print(action.shape)
        #print(done.shape)
        
        
        self.nn.eval()
        self.target_nn.eval()
            
        new_state_values=self.target_nn(new_state).detach()
        #print('shape of: new_state_values')
        #print(new_state_values.shape)
        
        max_new_state_values=torch.max(new_state_values,dim=1)[0]
        #print('shape of: max_new_state_values')
        #print(max_new_state_values.shape)
        target_value=(reward + (1-done)*gamma*max_new_state_values).view(-1,1)
        
        #print('shape of: target_value')
        #print(target_value.shape)
        self.nn.train()
        
        #this will select only the values of the desired actions
        predicted_value=torch.gather(self.nn(state),1,action)
        #print('shape of: predicted_value')
        #print(predicted_value.shape)
        
        
        loss=self.loss_function(predicted_value,target_value)
        self.optimizer.zero_grad()
        loss.backward()
        
        if clip_error:
            for param in self.nn.parameters():
                param.grad.clamp_(-1.0,1.0)
        
        self.optimizer.step()
        
        if self.update_target_counter % update_target_frequency == 0:
            #print("***********************")
            #print("UPDATING TARGET NETWORK")
            #print("update counter: {}".format(self.update_target_counter))
            #print("***********************")
            self.target_nn.load_state_dict(self.nn.state_dict())
        
        self.update_target_counter+=1
        

In [16]:
a=torch.randn([3,3])
print(a)

tensor([[-1.6049, -1.0776,  0.2535],
        [ 1.6732, -1.1906, -0.7068],
        [ 1.3573, -0.8768, -1.5517]])


In [17]:
a.clamp_(-1.0,1.0)

tensor([[-1.0000, -1.0000,  0.2535],
        [ 1.0000, -1.0000, -0.7068],
        [ 1.0000, -0.8768, -1.0000]])

In [18]:
a

tensor([[-1.0000, -1.0000,  0.2535],
        [ 1.0000, -1.0000, -0.7068],
        [ 1.0000, -0.8768, -1.0000]])

In [19]:
memory=ExperienceReplay(replay_memory_size)

In [20]:
qnet_agent=QNet_Agent()

In [21]:
seed_value=23
env.seed(seed_value)
torch.manual_seed(seed_value)
random.seed(seed_value)

In [ ]:
%%time

steps_total=np.full([num_episodes],-999,dtype=np.int32)
reward_total=np.full([num_episodes],-999,dtype=np.int32)

frames_total=0

solved_after = 0
solved = False

start_time = time.time()

for i_episode in range(num_episodes):
    
    state = env.reset()
    #for step in range(100):
    step=0
    reward_total[i_episode]=0
    
    while True:
        
        step+=1
        frames_total += 1
        
        epsilon=calculate_epsilon(frames_total)
        
        #action=env.action_space.sample()
        action=qnet_agent.select_action(state,epsilon)
        
        new_state, reward, done, info = env.step(action)
        memory.push(state, action, new_state,
                     reward, done)
        
        reward_total[i_episode]+=reward
        
        qnet_agent.optimize()
        
        state=new_state
        
        if done:
            steps_total[i_episode]=step
            
            if i_episode>100:
                mean_reward_100 = np.sum(reward_total[i_episode-100:i_episode])/100
            
                if (mean_reward_100 > score_to_solve and solved == False):
                    print("SOLVED! After %i episodes " % i_episode)
                    solved_after = i_episode
                    solved = True
            
            if (i_episode % report_interval == 0 and i_episode>1):
                print("**** Episode  {} **** ".format(i_episode))
                recent_avg_reward=np.average(reward_total[i_episode-report_interval:i_episode])
                print("Recent average reward: {}".format(recent_avg_reward))
                if i_episode>100:
                    print("Reward over last 100: {}".format(mean_reward_100))
                full_avg_so_far=np.average(reward_total[:i_episode])
                print("Average over all episodes so far: {}".format(full_avg_so_far))
                print("epsilon: {}".format(epsilon))
            
                #print("Episode {} finished after: {}".format(i_episode,step))
            break
            
if solved:
    print("Solved after %i episodes" % solved_after)
        

**** Episode  10 **** 
Recent average reward: 0.9
Average over all episodes so far: 0.9
epsilon: 0.8668487906593241
**** Episode  20 **** 
Recent average reward: 0.4
Average over all episodes so far: 0.65
epsilon: 0.8418078484171984
**** Episode  30 **** 
Recent average reward: 1.2
Average over all episodes so far: 0.8333333333333334
epsilon: 0.8134230906913473
**** Episode  40 **** 
Recent average reward: 0.9
Average over all episodes so far: 0.85
epsilon: 0.7868920917963156
**** Episode  50 **** 
Recent average reward: 1.7
Average over all episodes so far: 1.02
epsilon: 0.7535391811224643
**** Episode  60 **** 
Recent average reward: 1.3
Average over all episodes so far: 1.0666666666666667
epsilon: 0.7280372606275634
**** Episode  70 **** 
Recent average reward: 2.0
Average over all episodes so far: 1.2
epsilon: 0.6977060286799767
**** Episode  80 **** 
Recent average reward: 1.2
Average over all episodes so far: 1.2
epsilon: 0.6702123084260773
**** Episode  90 **** 
Recent average r

**** Episode  570 **** 
Recent average reward: 1.4
Reward over last 100: 1.52
Average over all episodes so far: 1.4736842105263157
epsilon: 0.10279766359769003
**** Episode  580 **** 
Recent average reward: 1.5
Reward over last 100: 1.61
Average over all episodes so far: 1.4741379310344827
epsilon: 0.09892038890404693
**** Episode  590 **** 
Recent average reward: 1.0
Reward over last 100: 1.57
Average over all episodes so far: 1.4661016949152543
epsilon: 0.09547136971458488
**** Episode  600 **** 
Recent average reward: 1.2
Reward over last 100: 1.48
Average over all episodes so far: 1.4616666666666667
epsilon: 0.09220872686980197
**** Episode  610 **** 
Recent average reward: 1.4
Reward over last 100: 1.46
Average over all episodes so far: 1.460655737704918
epsilon: 0.0888448078446117
**** Episode  620 **** 
Recent average reward: 1.9
Reward over last 100: 1.5
Average over all episodes so far: 1.467741935483871
epsilon: 0.08508354768948673
**** Episode  630 **** 
Recent average rewar

**** Episode  1090 **** 
Recent average reward: 2.0
Reward over last 100: 1.61
Average over all episodes so far: 1.5128440366972478
epsilon: 0.019984575605165533
**** Episode  1100 **** 
Recent average reward: 0.7
Reward over last 100: 1.58
Average over all episodes so far: 1.5054545454545454
epsilon: 0.01961708748620693
**** Episode  1110 **** 
Recent average reward: 1.3
Reward over last 100: 1.58
Average over all episodes so far: 1.5036036036036036
epsilon: 0.019215634673689787
**** Episode  1120 **** 
Recent average reward: 1.7
Reward over last 100: 1.57
Average over all episodes so far: 1.5053571428571428
epsilon: 0.018831999763226852
**** Episode  1130 **** 
Recent average reward: 1.5
Reward over last 100: 1.58
Average over all episodes so far: 1.5053097345132744
epsilon: 0.018459257998395418
**** Episode  1140 **** 
Recent average reward: 1.4
Reward over last 100: 1.6
Average over all episodes so far: 1.5043859649122806
epsilon: 0.01811538353824268
**** Episode  1150 **** 
Recent

**** Episode  1600 **** 
Recent average reward: 1.6
Reward over last 100: 1.44
Average over all episodes so far: 1.530625
epsilon: 0.011083548672706223
**** Episode  1610 **** 
Recent average reward: 2.8
Reward over last 100: 1.6
Average over all episodes so far: 1.5385093167701864
epsilon: 0.011030229365833586
**** Episode  1620 **** 
Recent average reward: 1.6
Reward over last 100: 1.62
Average over all episodes so far: 1.538888888888889
epsilon: 0.010983990991182812
**** Episode  1630 **** 
Recent average reward: 1.8
Reward over last 100: 1.66
Average over all episodes so far: 1.5404907975460123
epsilon: 0.010936862701583434
**** Episode  1640 **** 
Recent average reward: 2.4
Reward over last 100: 1.77
Average over all episodes so far: 1.545731707317073
epsilon: 0.010893437820915385
**** Episode  1650 **** 
Recent average reward: 1.6
Reward over last 100: 1.74
Average over all episodes so far: 1.5460606060606061
epsilon: 0.010854483114522799
**** Episode  1660 **** 
Recent average r

**** Episode  2110 **** 
Recent average reward: 1.4
Reward over last 100: 1.83
Average over all episodes so far: 1.5876777251184835
epsilon: 0.010108158391609341
**** Episode  2120 **** 
Recent average reward: 1.4
Reward over last 100: 1.75
Average over all episodes so far: 1.5867924528301887
epsilon: 0.01010377620875429
**** Episode  2130 **** 
Recent average reward: 1.7
Reward over last 100: 1.72
Average over all episodes so far: 1.587323943661972
epsilon: 0.010099309053658967
**** Episode  2140 **** 
Recent average reward: 2.3
Reward over last 100: 1.76
Average over all episodes so far: 1.5906542056074766
epsilon: 0.010094325988140477
**** Episode  2150 **** 
Recent average reward: 0.9
Reward over last 100: 1.69
Average over all episodes so far: 1.5874418604651164
epsilon: 0.010090533209727524
**** Episode  2160 **** 
Recent average reward: 1.8
Reward over last 100: 1.65
Average over all episodes so far: 1.588425925925926
epsilon: 0.010086889460489258
**** Episode  2170 **** 
Recent

...

**** Episode  46940 **** 
Recent average reward: 2.2
Reward over last 100: 1.72
Average over all episodes so far: 1.7484661269706008
epsilon: 0.01
**** Episode  46950 **** 
Recent average reward: 1.4
Reward over last 100: 1.68
Average over all episodes so far: 1.74839190628328
epsilon: 0.01
**** Episode  46960 **** 
Recent average reward: 1.4
Reward over last 100: 1.65
Average over all episodes so far: 1.7483177172061328
epsilon: 0.01
**** Episode  46970 **** 
Recent average reward: 1.4
Reward over last 100: 1.68
Average over all episodes so far: 1.7482435597189696
epsilon: 0.01
**** Episode  46980 **** 
Recent average reward: 1.8
Reward over last 100: 1.7
Average over all episodes so far: 1.748254576415496
epsilon: 0.01
**** Episode  46990 **** 
Recent average reward: 1.9
Reward over last 100: 1.67
Average over all episodes so far: 1.748286869546712
epsilon: 0.01
**** Episode  47000 **** 
Recent average reward: 2.3
Reward over last 100: 1.74
Average over all episodes so far: 1.7484042

**** Episode  47500 **** 
Recent average reward: 1.6
Reward over last 100: 1.53
Average over all episodes so far: 1.7477473684210527
epsilon: 0.01
**** Episode  47510 **** 
Recent average reward: 1.7
Reward over last 100: 1.62
Average over all episodes so far: 1.7477373184592717
epsilon: 0.01
**** Episode  47520 **** 
Recent average reward: 1.9
Reward over last 100: 1.68
Average over all episodes so far: 1.7477693602693603
epsilon: 0.01
**** Episode  47530 **** 
Recent average reward: 1.2
Reward over last 100: 1.66
Average over all episodes so far: 1.7476541131916685
epsilon: 0.01
**** Episode  47540 **** 
Recent average reward: 1.6
Reward over last 100: 1.7
Average over all episodes so far: 1.7476230542700883
epsilon: 0.01
**** Episode  47550 **** 
Recent average reward: 1.8
Reward over last 100: 1.69
Average over all episodes so far: 1.747634069400631
epsilon: 0.01
**** Episode  47560 **** 
Recent average reward: 1.7
Reward over last 100: 1.71
Average over all episodes so far: 1.7476

**** Episode  48060 **** 
Recent average reward: 1.1
Reward over last 100: 1.5
Average over all episodes so far: 1.7465875988347899
epsilon: 0.01
**** Episode  48070 **** 
Recent average reward: 2.2
Reward over last 100: 1.61
Average over all episodes so far: 1.7466819221967964
epsilon: 0.01
**** Episode  48080 **** 
Recent average reward: 1.1
Reward over last 100: 1.61
Average over all episodes so far: 1.7465474209650582
epsilon: 0.01
**** Episode  48090 **** 
Recent average reward: 2.2
Reward over last 100: 1.66
Average over all episodes so far: 1.7466417134539405
epsilon: 0.01
**** Episode  48100 **** 
Recent average reward: 2.8
Reward over last 100: 1.77
Average over all episodes so far: 1.746860706860707
epsilon: 0.01
**** Episode  48110 **** 
Recent average reward: 2.2
Reward over last 100: 1.79
Average over all episodes so far: 1.7469548950322178
epsilon: 0.01
**** Episode  48120 **** 
Recent average reward: 1.2
Reward over last 100: 1.74
Average over all episodes so far: 1.7468

**** Episode  48620 **** 
Recent average reward: 1.3
Reward over last 100: 1.84
Average over all episodes so far: 1.745392842451666
epsilon: 0.01
**** Episode  48630 **** 
Recent average reward: 1.4
Reward over last 100: 1.8
Average over all episodes so far: 1.7453218178079375
epsilon: 0.01
**** Episode  48640 **** 
Recent average reward: 0.8
Reward over last 100: 1.74
Average over all episodes so far: 1.7451274671052632
epsilon: 0.01
**** Episode  48650 **** 
Recent average reward: 2.1
Reward over last 100: 1.7
Average over all episodes so far: 1.7452004110996917
epsilon: 0.01
**** Episode  48660 **** 
Recent average reward: 1.6
Reward over last 100: 1.7
Average over all episodes so far: 1.7451705713111385
epsilon: 0.01
**** Episode  48670 **** 
Recent average reward: 2.1
Reward over last 100: 1.68
Average over all episodes so far: 1.7452434764742142
epsilon: 0.01
**** Episode  48680 **** 
Recent average reward: 1.8
Reward over last 100: 1.71
Average over all episodes so far: 1.745254

**** Episode  49180 **** 
Recent average reward: 1.5
Reward over last 100: 1.84
Average over all episodes so far: 1.7456689711264741
epsilon: 0.01
**** Episode  49190 **** 
Recent average reward: 1.6
Reward over last 100: 1.8
Average over all episodes so far: 1.7456393575930067
epsilon: 0.01
**** Episode  49200 **** 
Recent average reward: 1.7
Reward over last 100: 1.76
Average over all episodes so far: 1.745630081300813
epsilon: 0.01
**** Episode  49210 **** 
Recent average reward: 1.7
Reward over last 100: 1.69
Average over all episodes so far: 1.7456208087787035
epsilon: 0.01
**** Episode  49220 **** 
Recent average reward: 2.5
Reward over last 100: 1.81
Average over all episodes so far: 1.745774075579033
epsilon: 0.01
**** Episode  49230 **** 
Recent average reward: 1.3
Reward over last 100: 1.79
Average over all episodes so far: 1.7456835263050985
epsilon: 0.01
**** Episode  49240 **** 
Recent average reward: 2.4
Reward over last 100: 1.85
Average over all episodes so far: 1.74581

In [ ]:
plt.figure(2,figsize=[12,5])
plt.title("Score at end of episode")
plt.plot(reward_total[:frames_total])
plt.show()

In [ ]:
print("Average number of steps: {}". format(np.average(steps_total[:frames_total])))

In [ ]:
print("Average number of steps=reward: {}". format(np.average(steps_total)))
print("Average reward in last 100 episodes: {}". format(np.average(reward_total[frames_total-100:frames_total])))



plt.figure(1,figsize=[12,5])
plt.title("Rewards")
plt.bar(torch.arange(len(rewards_total)), rewards_total,alpha=0.6, color='green')
#plt.plot(rewards_total)
plt.show()

In [ ]:
plt.figure(2,figsize=[12,5])
plt.title("Steps to finish episode")
plt.plot(steps_total[:frames_total])
plt.show()